# Exploring ICS ATT&CK

## Query ATT&CK

### Import TAXII Libraries
ATT&CK users can use the initial Server class to instantiate a server object pointing to the framework’s public TAXII server URL https://cti-taxii.mitre.org/taxii/

In [1]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)

In [2]:
server = Server("https://cti-taxii.mitre.org/taxii/")

Available API Roots can be referenced from the server object. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API:

In [3]:
server.api_roots

In [4]:
api_root = server.api_roots[0]

### Explore ATT&CK TAXII Collections
The **collections** attribute can then be used and get more information about them via their respective available properties:

In [5]:
api_root.collections

In [6]:
for collection in api_root.collections:
    print(collection.title, "->", collection.description)

Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


In [7]:
api_root.collections[3].title

'ICS ATT&CK'

In [8]:
api_root.collections[3].id

'02c3ef24-9cd4-48f3-a99f-b74ce24f1d34'

## Set ICS ATT&CK TAXII Collection ID Variable

In [9]:
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [10]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

In [11]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Retrieve all ICS Techniques
Now that we can query the ICS ATT&CK TAXIICollection. We can use the query method and a set of filter to retrieve STIX objects of type "attack-pattern" -> "Techniques"

In [12]:
ICS_TECHNIQUES = TC_ICS_SOURCE.query(Filter("type", "=", "attack-pattern"))
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', id='attack-pattern--77d9c726-b53e-481d-8bcc-1068aebfbb9d', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2024-03-26T15:39:19.473Z', modified='2024-04-08T18:54:40.925Z', name='Autorun Image', description='Adversaries may leverage AutoRun functionality or scripts to execute malicious code. Devices configured to enable AutoRun functionality or legacy operating systems may be susceptible to abuse of these features to run malicious code stored on various forms of removeable media (i.e., USB, Disk Images [.ISO]). Commonly, AutoRun or AutoPlay are disabled in many operating systems configurations to mitigate against this technique. If a device is configured to enable AutoRun or AutoPlay, adversaries may execute code on the device by mounting the removable media to the device, either through physical or virtual means. This may be especially relevant for virtual machine environments where disk images may be dynamically mapped to a

In [13]:
for TECHNIQUE in ICS_TECHNIQUES:
    print(TECHNIQUE['external_references'][0]['external_id'], "--", TECHNIQUE['name'])

T0895 -- Autorun Image
T0894 -- System Binary Proxy Execution
T0893 -- Data from Local System
T0892 -- Change Credential
T0891 -- Hardcoded Credentials
T0864 -- Transient Cyber Asset
T0888 -- Remote System Information Discovery
T0834 -- Native API
T0890 -- Exploitation for Privilege Escalation
T0821 -- Modify Controller Tasking
T0889 -- Modify Program
T0886 -- Remote Services
T0837 -- Loss of Protection
T0881 -- Service Stop
T0800 -- Activate Firmware Update Mode
T0850 -- Role Identification
T0860 -- Wireless Compromise
T0801 -- Monitor Process State
T0811 -- Data from Information Repositories
T0802 -- Automated Collection
T0810 -- Data Historian Compromise
T0871 -- Execution through API
T0865 -- Spearphishing Attachment
T0856 -- Spoof Reporting Message
T0870 -- Detect Program State
T0874 -- Hooking
T0843 -- Program Download
T0833 -- Modify Control Logic
T0885 -- Commonly Used Port
T0883 -- Internet Accessible Device
T0836 -- Modify Parameter
T0831 -- Manipulation of Control
T0807 -- C

## ICS ATT&CK Available since attackcti 0.3.4.3
Reference: https://pypi.org/project/attackcti/

In [14]:
from attackcti import attack_client
lift = attack_client()

ICS_TECHNIQUES = lift.get_ics_techniques()
print("Techniques Count:",len(ICS_TECHNIQUES))

Techniques Count: 83


In [15]:
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', id='attack-pattern--77d9c726-b53e-481d-8bcc-1068aebfbb9d', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2024-03-26T15:39:19.473Z', modified='2024-04-08T18:54:40.925Z', name='Autorun Image', description='Adversaries may leverage AutoRun functionality or scripts to execute malicious code. Devices configured to enable AutoRun functionality or legacy operating systems may be susceptible to abuse of these features to run malicious code stored on various forms of removeable media (i.e., USB, Disk Images [.ISO]). Commonly, AutoRun or AutoPlay are disabled in many operating systems configurations to mitigate against this technique. If a device is configured to enable AutoRun or AutoPlay, adversaries may execute code on the device by mounting the removable media to the device, either through physical or virtual means. This may be especially relevant for virtual machine environments where disk images may be dynamically mapped to a

## Get All Data Sources Mapped to ICS ATT&CK Techniques

In [16]:
ICS_DATA_SOURCES = []
for TECHNIQUE in ICS_TECHNIQUES:
    if 'x_mitre_data_sources' in TECHNIQUE.keys():
        for DS in TECHNIQUE['x_mitre_data_sources']:
            if DS not in ICS_DATA_SOURCES:
                ICS_DATA_SOURCES.append(DS)
ICS_DATA_SOURCES

['Drive: Drive Creation',
 'Process: Process Creation',
 'Script: Script Execution',
 'Command: Command Execution',
 'File: File Access',
 'Process: OS API Execution',
 'Operational Databases: Device Alarm',
 'Network Traffic: Network Traffic Content',
 'Logon Session: Logon Session Creation',
 'Network Traffic: Network Traffic Flow',
 'Application Log: Application Log Content',
 'Asset: Software',
 'Module: Module Load',
 'Network Traffic: Network Connection Creation',
 'Network Share: Network Share Access',
 'File: File Modification',
 'Process: Process Termination',
 'Service: Service Metadata',
 'Windows Registry: Windows Registry Key Modification',
 'File: File Creation',
 'Process: Process Metadata',
 'Asset: Asset Inventory',
 'Logon Session: Logon Session Metadata',
 'Operational Databases: Process/Event Alarm',
 'Operational Databases: Process History/Live Data',
 'File: File Metadata',
 'Service: Service Creation',
 'Windows Registry: Windows Registry Key Deletion',
 'File: F

## Get All Groups from ICS ATT&CK

In [17]:
ICS_GROUPS = lift.get_ics_groups()
for GROUP in ICS_GROUPS:
    print(GROUP['name'])

CyberAv3ngers
GOLD SOUTHFIELD
Wizard Spider
TEMP.Veles
APT38
HEXANE
APT33
OilRig
FIN7
FIN6
Dragonfly
Sandworm Team
Lazarus Group
ALLANITE


## Get All Malware from ICS ATT&CK

In [18]:
ICS_MALWARE = lift.get_ics_malware()
for MALWARE in ICS_MALWARE:
    print(MALWARE['name'])

Industroyer2
INCONTROLLER
EKANS
Conficker
EKANS
Bad Rabbit
KillDisk
Industroyer
Stuxnet
REvil
Ryuk
LockerGoga
NotPetya
VPNFilter
Stuxnet
Triton
PLC-Blaster
WannaCry
Flame
Industroyer
Killdisk
Backdoor.Oldrea
ACAD/Medre.A
Bad Rabbit
Conficker
BlackEnergy 3
BlackEnergy
Duqu
